In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Steps :
1. Load Dataset
2. MakeData set iterable
3. Create a Model class
4. Instantiate a Model Class
5. Instantiate a Loss class
6. Instantiate a Optimizer Class
7. Train Model

In [ ]:
# Step 1 :  Load the MNIST Train Dataset

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [ ]:
train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True
                            )

test_dataset = dsets.MNIST(root="./data",
                           train=False,
                           transform=transforms.ToTensor())

  0%|          | 0/9912422 [00:00<?, ?it/s]

9920512it [00:00, 20734273.22it/s]                            


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


32768it [00:00, 299203.77it/s]                           
0it [00:00, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


1654784it [00:00, 5631034.85it/s]                           
8192it [00:00, 121039.97it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [ ]:
#Step 2 :  Make dataset Iterable 
batch_size=100
n_iters = 3000
n_epochs = n_iters / (len(train_dataset)/batch_size)
n_epochs = int(n_epochs)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=batch_size,
                                              shuffle=False)

In [ ]:
# Step 3 : Create a Model Class 

class FeedforwardNNModelSIG(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
      super().__init__()
      
      #Linear Layar
      self.fc1=nn.Linear(input_dim,hidden_dim)

      #Non Linear Layaer 
      self.sigmoid=nn.Sigmoid()

      #Linear Layar(readout layer)
      self.fc2=nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
      # Linear
      out = self.fc1(x)
      # Non-Linear
      out = self.sigmoid(out)
      # Linear (readout layer)
      out = self.fc2(out)

      return out


In [ ]:
# Step 4 : Intantiate Model Class

input_dim=28*28
hidden_dim=50
output_dim=10
model = FeedforwardNNModelSIG(input_dim,hidden_dim,output_dim)

In [ ]:
# Step 5 : Intantiate loss class
criterion = nn.CrossEntropyLoss()

In [ ]:
# Step 6 : Intantiate optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
# Step 7 : Train the Model
##1 Convert inputs/labels to Variables
##2 Clear the gradients 
##3 Get output given input
##4 Get Loss
##5 Get gradients wrt parameters
##6 Update paramters using Gradients
##7 Repeat

iter=0
for epoch in range(n_epochs):
  for i,(images, labels) in enumerate(train_dataloader):

      ##1 Convert inputs/labels to Variable
      images = Variable(images.view(-1,28*28))
      labels = Variable(labels)

      ##2 Clear the gradients 
      optimizer.zero_grad()

      ##3 Get output given input
      outputs = model(images)

      ##4 Get Loss
      loss = criterion(outputs,labels)

      ##5 Get gradients wrt parameters
      loss.backward()
    
      ##6 Update paramters using Gradients
      optimizer.step()

      ##7 Repeat

      iter +=1
      if iter % 500 == 0 :
        correct = 0.
        total = 0.
      # Iter thorough test dataset
        for images,labels in test_dataloader:
          images = Variable(images.view(-1,28*28))
          labels = Variable(labels)

          outputs = model(images)
          # Get the max value of the prediction
          _,predicted=torch.max(outputs.data,1)
          # number of labels 
          total += labels.size(0)
          # Total correct predictions
          correct += (predicted==labels).sum()

        accuracy = 100*(correct/total)
        print("Iteration: {} , Loss:{} , Accuracy:{}".format(iter,loss.data,accuracy))

Iteration: 500 , Loss:0.6747167706489563 , Accuracy:85.06999969482422
Iteration: 1000 , Loss:0.4159547686576843 , Accuracy:89.49000549316406
Iteration: 1500 , Loss:0.30652761459350586 , Accuracy:90.37000274658203
Iteration: 2000 , Loss:0.4730711877346039 , Accuracy:91.08000183105469
Iteration: 2500 , Loss:0.1980995535850525 , Accuracy:91.5199966430664
Iteration: 3000 , Loss:0.2555803060531616 , Accuracy:91.97999572753906


In [ ]:
def print_graph(g, level=0):
    if g == None: return
    print('#'*level*1, g)
    for subg in g.next_functions:
        #print(subg)
        print_graph(subg[0], level+1)

print_graph(loss.grad_fn, 1)

# <NllLossBackward object at 0x7f61cbdcf5f8>
## <LogSoftmaxBackward object at 0x7f61cbdc8a90>
### <AddmmBackward object at 0x7f61cbdcf748>
#### <AccumulateGrad object at 0x7f61cbdcf7f0>
#### <SigmoidBackward object at 0x7f61cbdcf828>
##### <AddmmBackward object at 0x7f61cbdcf940>
###### <AccumulateGrad object at 0x7f61cbdcf9e8>
###### <TBackward object at 0x7f61cbdcfa20>
####### <AccumulateGrad object at 0x7f61cbdcfb00>
#### <TBackward object at 0x7f61cbdcf860>
##### <AccumulateGrad object at 0x7f61cbdcf978>


In [ ]:
# Step 3 : Create a Model Class 

class FeedforwardNNModelTANH(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
      super().__init__()
      
      #Linear Layar
      self.fc1=nn.Linear(input_dim,hidden_dim)

      #Non Linear Layaer 
      self.tanh=nn.Tanh()

      #Linear Layar(readout layer)
      self.fc2=nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
      # Linear
      out = self.fc1(x)
      # Non-Linear
      out = self.tanh(out)
      # Linear (readout layer)
      out = self.fc2(out)

      return out

In [ ]:
# Step 4 : Intantiate Model Class

input_dim=28*28
hidden_dim=100
output_dim=10
model = FeedforwardNNModelTANH(input_dim,hidden_dim,output_dim)

In [ ]:
# Step 5 : Intantiate loss class
criterion = nn.CrossEntropyLoss()

In [ ]:
# Step 6 : Intantiate optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
# Step 7 : Train the Model
##1 Convert inputs/labels to Variables
##2 Clear the gradients 
##3 Get output given input
##4 Get Loss
##5 Get gradients wrt parameters
##6 Update paramters using Gradients
##7 Repeat

iter=0
for epoch in range(n_epochs):
  for i,(images, labels) in enumerate(train_dataloader):

      ##1 Convert inputs/labels to Variable
      images = Variable(images.view(-1,28*28))
      labels = Variable(labels)

      ##2 Clear the gradients 
      optimizer.zero_grad()

      ##3 Get output given input
      outputs = model(images)

      ##4 Get Loss
      loss = criterion(outputs,labels)

      ##5 Get gradients wrt parameters
      loss.backward()
    
      ##6 Update paramters using Gradients
      optimizer.step()

      ##7 Repeat

      iter +=1
      if iter % 500 == 0 :
        correct = 0.
        total = 0.
      # Iter thorough test dataset
        for images,labels in test_dataloader:
          images = Variable(images.view(-1,28*28))
          labels = Variable(labels)

          outputs = model(images)
          # Get the max value of the prediction
          _,predicted=torch.max(outputs.data,1)
          # number of labels 
          total += labels.size(0)
          # Total correct predictions
          correct += (predicted==labels).sum()

        accuracy = 100*(correct/total)
        print("Iteration: {} , Loss:{} , Accuracy:{}".format(iter,loss.data,accuracy))

Iteration: 500 , Loss:0.336751788854599 , Accuracy:91.08000183105469
Iteration: 1000 , Loss:0.3685636520385742 , Accuracy:92.62999725341797
Iteration: 1500 , Loss:0.32898658514022827 , Accuracy:93.23999786376953
Iteration: 2000 , Loss:0.25302228331565857 , Accuracy:94.0
Iteration: 2500 , Loss:0.20827388763427734 , Accuracy:94.77999877929688
Iteration: 3000 , Loss:0.2247171849012375 , Accuracy:95.18000030517578


In [ ]:
print_graph(loss.grad_fn, 1)

# <NllLossBackward object at 0x7f61cbdc8a20>
## <LogSoftmaxBackward object at 0x7f61cbdc87b8>
### <AddmmBackward object at 0x7f61cbdc8c88>
#### <AccumulateGrad object at 0x7f61cbdc80f0>
#### <TanhBackward object at 0x7f61cbdc8198>
##### <AddmmBackward object at 0x7f61cbdc80b8>
###### <AccumulateGrad object at 0x7f61cbdc8278>
###### <TBackward object at 0x7f61cbdc82b0>
####### <AccumulateGrad object at 0x7f61cbdc8208>
#### <TBackward object at 0x7f61cbdc8080>
##### <AccumulateGrad object at 0x7f61cbdc81d0>


In [ ]:
# Step 3 : Create a Model Class 

class FeedforwardNNModelRelu(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
      super().__init__()
      
      #Linear Layar
      self.fc1=nn.Linear(input_dim,hidden_dim)

      #Non Linear Layaer 
      self.relu=nn.ReLU()

      #Linear Layar(readout layer)
      self.fc2=nn.Linear(hidden_dim,output_dim)

  def forward(self,x):
      # Linear
      out = self.fc1(x)
      # Non-Linear
      out = self.relu(out)
      # Linear (readout layer)
      out = self.fc2(out)

      return out

In [ ]:
# Step 4 : Intantiate Model Class

input_dim=28*28
hidden_dim=100
output_dim=10
model = FeedforwardNNModelRelu(input_dim,hidden_dim,output_dim)

In [ ]:
# Step 5 : Intantiate loss class
criterion = nn.CrossEntropyLoss()

In [ ]:
# Step 6 : Intantiate optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
# Step 7 : Train the Model
##1 Convert inputs/labels to Variables
##2 Clear the gradients 
##3 Get output given input
##4 Get Loss
##5 Get gradients wrt parameters
##6 Update paramters using Gradients
##7 Repeat

iter=0
for epoch in range(n_epochs):
  for i,(images, labels) in enumerate(train_dataloader):

      ##1 Convert inputs/labels to Variable
      images = Variable(images.view(-1,28*28))
      labels = Variable(labels)

      ##2 Clear the gradients 
      optimizer.zero_grad()

      ##3 Get output given input
      outputs = model(images)

      ##4 Get Loss
      loss = criterion(outputs,labels)

      ##5 Get gradients wrt parameters
      loss.backward()
    
      ##6 Update paramters using Gradients
      optimizer.step()

      ##7 Repeat

      iter +=1
      if iter % 500 == 0 :
        correct = 0.
        total = 0.
      # Iter thorough test dataset
        for images,labels in test_dataloader:
          images = Variable(images.view(-1,28*28))
          labels = Variable(labels)

          outputs = model(images)
          # Get the max value of the prediction
          _,predicted=torch.max(outputs.data,1)
          # number of labels 
          total += labels.size(0)
          # Total correct predictions
          correct += (predicted==labels).sum()

        accuracy = 100*(correct/total)
        print("Iteration: {} , Loss:{} , Accuracy:{}".format(iter,loss.data,accuracy))

Iteration: 500 , Loss:0.27933087944984436 , Accuracy:91.70999908447266
Iteration: 1000 , Loss:0.4297708570957184 , Accuracy:93.19000244140625
Iteration: 1500 , Loss:0.2575586140155792 , Accuracy:93.95999908447266
Iteration: 2000 , Loss:0.2052202969789505 , Accuracy:94.76000213623047
Iteration: 2500 , Loss:0.25132447481155396 , Accuracy:95.42000579833984
Iteration: 3000 , Loss:0.1009683907032013 , Accuracy:95.63999938964844


In [ ]:
# Step 3 : Create a Model Class multiple hidden

class FeedforwardNNModelMRelu(nn.Module):
  def __init__(self,input_dim,hidden_dim,output_dim):
      super().__init__()
      
      #Linear Layer
      self.fc1=nn.Linear(input_dim,hidden_dim)

      #Non Linear Layer 
      self.relu1=nn.ReLU()

      #Linear Layer(readout layer)
      self.fc2=nn.Linear(hidden_dim,hidden_dim)

      #Non Linear Layer 
      self.relu2=nn.ReLU()

      #Linear Layer(readout layer)
      self.fc3=nn.Linear(hidden_dim,hidden_dim)

      #Non Linear Layer 
      self.relu3=nn.ReLU()

      #Linear Layer
      self.fc4=nn.Linear(hidden_dim,hidden_dim)

      #Non Linear Layer 
      self.relu4=nn.ReLU()

      #Linear Layer(readout layer)
      self.fc5=nn.Linear(hidden_dim,hidden_dim)

      #Non Linear Layer 
      self.relu5=nn.ReLU()

      #Linear Layer(readout layer)
      self.fc6=nn.Linear(hidden_dim,output_dim)




  def forward(self,x):

      out = self.fc1(x)
      out = self.relu1(out)
      
      out = self.fc2(out)
      out = self.relu2(out)
      
      out = self.fc3(out)
      out = self.relu3(out)

      out = self.fc4(out)
      out = self.relu4(out)

      out = self.fc5(out)
      out = self.relu5(out)
      
      # Linear(readout layer)
      out = self.fc6(out)

      return out

In [ ]:
# Step 4 : Intantiate Model Class

input_dim=28*28
hidden_dim=1000
output_dim=10
model = FeedforwardNNModelMRelu(input_dim,hidden_dim,output_dim)

In [ ]:
# Step 5 : Intantiate loss class
criterion = nn.CrossEntropyLoss()

In [ ]:
# Step 6 : Intantiate optimizer
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
# Step 7 : Train the Model
##1 Convert inputs/labels to Variables
##2 Clear the gradients 
##3 Get output given input
##4 Get Loss
##5 Get gradients wrt parameters
##6 Update paramters using Gradients
##7 Repeat

iter=0
for epoch in range(n_epochs):
  for i,(images, labels) in enumerate(train_dataloader):

      ##1 Convert inputs/labels to Variable
      images = Variable(images.view(-1,28*28))
      labels = Variable(labels)

      ##2 Clear the gradients 
      optimizer.zero_grad()

      ##3 Get output given input
      outputs = model(images)

      ##4 Get Loss
      loss = criterion(outputs,labels)

      ##5 Get gradients wrt parameters
      loss.backward()
    
      ##6 Update paramters using Gradients
      optimizer.step()

      ##7 Repeat

      iter +=1
      if iter % 500 == 0 :
        correct = 0.
        total = 0.
      # Iter thorough test dataset
        for images,labels in test_dataloader:
          images = Variable(images.view(-1,28*28))
          labels = Variable(labels)

          outputs = model(images)
          # Get the max value of the prediction
          _,predicted=torch.max(outputs.data,1)
          # number of labels 
          total += labels.size(0)
          # Total correct predictions
          correct += (predicted==labels).sum()

        accuracy = 100*(correct/total)
        print("Iteration: {} , Loss:{} , Accuracy:{}".format(iter,loss.data,accuracy))

Iteration: 500 , Loss:0.5044265985488892 , Accuracy:86.80999755859375
Iteration: 1000 , Loss:0.12980115413665771 , Accuracy:94.22000122070312
Iteration: 1500 , Loss:0.08640319108963013 , Accuracy:96.21000671386719
Iteration: 2000 , Loss:0.05886375904083252 , Accuracy:96.13999938964844
Iteration: 2500 , Loss:0.07721692323684692 , Accuracy:97.25
Iteration: 3000 , Loss:0.13240040838718414 , Accuracy:97.40999603271484


In [ ]:
print_graph(loss.grad_fn, 1)

# <NllLossBackward object at 0x7f61cbdc8048>
## <LogSoftmaxBackward object at 0x7f61cbd9dc18>
### <AddmmBackward object at 0x7f61cbd9d908>
#### <AccumulateGrad object at 0x7f61cbd9d710>
#### <ReluBackward0 object at 0x7f61cbd9da90>
##### <AddmmBackward object at 0x7f61cbdcfac8>
###### <AccumulateGrad object at 0x7f61cbdcfa20>
###### <ReluBackward0 object at 0x7f61cbdcf128>
####### <AddmmBackward object at 0x7f61cbdcf198>
######## <AccumulateGrad object at 0x7f61cbdcfb70>
######## <ReluBackward0 object at 0x7f61cbdcf588>
######### <AddmmBackward object at 0x7f61cbdcfdd8>
########## <AccumulateGrad object at 0x7f61cbdcf4a8>
########## <ReluBackward0 object at 0x7f61cbc68048>
########### <AddmmBackward object at 0x7f61cbc682b0>
############ <AccumulateGrad object at 0x7f61cbdcfb38>
############ <ReluBackward0 object at 0x7f61cbdcfcc0>
############# <AddmmBackward object at 0x7f61cbdcf390>
############## <AccumulateGrad object at 0x7f622659c6d8>
############## <TBackward object at 0x7f61cb